# CAFE60_reduce_write_zarr

**Date:** <br>
16 March 2022 <br>
**Background:** <br>
Issue - https://github.com/Thomas-Moore-Creative/NCI-CAFE-ARD/issues/1 <br>
**Author(s):**<br>
Thomas Moore<br>

## We are using NCI OOD as platform for data processing
### OOD documentation
https://opus.nci.org.au/display/DAE/Setting+up+a+Dask+Cluster+on+OOD

In [1]:
Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

In [2]:
import xarray as xr
import numpy as np
import xrft
import xesmf as xe
import scipy
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import os
import re
import cartopy.crs as ccrs
import proplot as pplt
from rechunker import rechunk
%config Completer.use_jedi = False

## import helper

In [3]:
import importlib.util
spec = importlib.util.spec_from_file_location("helper", "/g/data/v14/tm4888/code/helper-py/helper_tools.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

## OOD cluster

In [4]:
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,processes=1,memory="47GB",walltime='03:00:00')
client = Client(cluster)
cluster.scale(cores=24)

/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.155:46319,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Load in CAFE60 zarr collection

In [33]:
ds_CAFE60 = xr.open_zarr('/g/data/xv83/dcfp/CAFE60v1/ocean_month.zarr.zip',consolidated=True)

In [34]:
ds_CAFE60.nbytes/1e9

30376.664081592

# 30 TB!

# drop variables not in use

In [35]:
ds_CAFE60 = ds_CAFE60.drop({'age_global','cfc_11','cfc_12','eta_t','neutral_diffusion_salt',
                            'neutral_diffusion_temp','neutral_gm_salt','neutral_gm_temp','salt_sponge_tend',
                            'salt_vdiff_impl','temp_sponge_tend','temp_vdiff_impl','tx_trans_gm','ty_trans_gm',
                            'wt','sw_ocean'})

In [36]:
ds_CAFE60.nbytes/1e9

9125.125649192

# 9 TB!

# Reduce time period to 1980

In [37]:
ds_CAFE60 = ds_CAFE60.isel(time=slice(240,730))

In [38]:
ds_CAFE60.nbytes/1e9

6116.705295264

# 6TB!

# Reduce to ensemble mean!

In [39]:
ds_CAFE60 = ds_CAFE60.mean(dim = 'ensemble', keep_attrs=True, skipna=True)

/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [41]:
ds_CAFE60.nbytes/1e9

63.71569488

# 63 GB $phew!?$


# rechunk to reduce small chunks

In [42]:
ds_CAFE60 = ds_CAFE60.chunk({'time':490})

# Can we write reduced zarr file at this size?

In [44]:
%%time
ds_CAFE60.to_zarr('/g/data/xv83/users/tm4888/data/CAFE/CAFE60_reduced_ocean_month.zarr',consolidated=True)

CPU times: user 33min 8s, sys: 1min 37s, total: 34min 46s
Wall time: 53min 7s


# Read back in and test

In [45]:
ds_test = xr.open_zarr('/g/data/xv83/users/tm4888/data/CAFE/CAFE60_reduced_ocean_month.zarr',consolidated=True)

In [47]:
ds_test

<xarray.Dataset>
Dimensions:   (time: 490, st_ocean: 50, yt_ocean: 300, xt_ocean: 360, xu_ocean: 360, yu_ocean: 300)
Coordinates:
  * st_ocean  (st_ocean) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
  * time      (time) object 1980-01-16 12:00:00 ... 2020-10-16 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * xu_ocean  (xu_ocean) float64 -279.0 -278.0 -277.0 -276.0 ... 78.0 79.0 80.0
  * yt_ocean  (yt_ocean) float64 -77.88 -77.63 -77.38 ... 88.87 89.32 89.77
  * yu_ocean  (yu_ocean) float64 -77.75 -77.51 -77.26 -77.01 ... 89.1 89.55 90.0
Data variables:
    salt      (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(490, 3, 300, 360), meta=np.ndarray>
    sst       (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(490, 300, 360), meta=np.ndarray>
    temp      (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(490, 3, 300, 360), meta=np.ndarray>
    tx_trans  (time, st_ocean, yt_ocean, xu_ocean) float32 dask.array<chunksize=(490, 3, 300, 360), meta=np.ndarray>
    ty_trans  (time, st_ocean, yu_ocean, xt_ocean) float32 dask.array<chunksize=(490, 3, 300, 360), meta=np.ndarray>
    u         (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(490, 3, 300, 360), meta=np.ndarray>
    v         (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(490, 3, 300, 360), meta=np.ndarray>
Attributes: (12/24)
    calendar:                 julian
    cm-enkf_source:          commit 0141830a243704acbf6d8c8c843b161a39ec39fa
    contact_name:            Decadal Activity 1 - Data Assimilation
    control_name:            c5
    creation_date:           Thu Nov  5 17:19:53 2020
    data_assimilation_name:  d60
    ...                      ...
    model_source:            commit 2abb29f4384e68777721fa657850587f28efd85a
    nominal_resoltuion:      Atmosphere delta lat = 2.02degrees ; Atmosphere ...
    perturbation_name:       not applicable
    references:              OKane, T.J., Sandery, P.A., Monselesan, D.P., Sa...
    run_variant_name:        data assimilation
    title:                   AccessOcean-AM2

# $ The\ End$

# Break glass in case of emergency
# $\Downarrow$

In [33]:
client.restart()

<Client: 'tcp://10.0.128.155:40685' processes=8 threads=16, memory=350.16 GiB>

In [49]:
client.shutdown()